In [1]:
import pandas as pd
import geopandas as gp
import folium
from folium import plugins
import glob
import numpy as np

In [5]:
year = 2019
month = 7

In [9]:
if (year == 2019) & (month in [7,9,10,11,12]):
    print('yes')
else:
    print('no')

yes


TODs
1 - 10pm-6am
2 - 6am-9am
3 - 9am-4pm
4 - 4pm-7pm
5 - 7pm-10pm

In [2]:
tods = [1,2,3,4,5]

df2 = pd.DataFrame()

for tod in tods:  
    path = r'C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/' + str(tod)  # use your path
    all_files = glob.glob(path + "/*.csv")


    for filename in all_files:
        print('Working on file ' + filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        
    df['TOD'] = tod
    df = df.groupby(by = ['origin', 'destination', 'TOD'], as_index = False).mean()
    df2 = df2.append(df)

    #frame = pd.concat(li, axis=0, ignore_index=True)

Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-1.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-11.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-22.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-34.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-42.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_0-54.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_1-1.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_1-19.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/1\traveltime_matrix_1-24.csv
Working on file C:/Workspace/T

Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_11-59.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_11-9.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-18.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-25.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-38.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-45.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-51.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_12-7.csv
Working on file C:/Workspace/TNC-Demand-Model/Outputs/OTP Travel Times/Transit/3\traveltime_matrix_13-1.csv
Working on file C:/Wor

In [3]:
geo = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [4]:
geo['geoid10'] = geo.geoid10.astype(float)

In [5]:
df2['TRAVEL_TIME'] = df2['travel_time']/60


In [6]:
centroids = pd.read_csv('C:/Workspace/TNC-Demand-Model/otp/points.csv')

centroids = gp.GeoDataFrame(centroids)
centroids.head()

,Unnamed: 0,X,Y,GEOID
0,0,-87.630040,41.742475,17031842400
1,1,-87.681882,41.832094,17031840300
2,2,-87.635098,41.851006,17031841100
3,3,-87.683342,41.855562,17031841200
4,4,-87.631405,41.871044,17031839000


In [7]:
df2.to_csv('C:/Workspace/TNC-Demand-Model/Outputs/Transit_Travel_Times.csv')

In [8]:
centroids['GEOID']  = centroids['GEOID'].astype(float)

In [9]:
len(df2)

3208005

In [10]:
for tod in tods:
    print('Working on TOD ' + str(tod))
    
    for tract in geo.geoid10:
        m = folium.Map([41.8781, -87.6298], zoom_start=11, tiles = 'Stamen Toner')
        
        folium.Marker([centroids[centroids['GEOID'] == tract]['Y'].values[0], centroids[centroids['GEOID'] == tract]['X'].values[0]] , tooltip='Origin').add_to(m)
        
        df_tod = df2[df2['TOD'] == tod]
        df_tract = df_tod[df_tod['origin'] == tract]

        bins = [0,15,30,60,90,df_tract.TRAVEL_TIME.max()]

        folium.Choropleth(
                 geo_data=geo,
                 name= str(tract),
                 data=df_tract,
                 columns = ['destination', 'TRAVEL_TIME'],
                 key_on='feature.properties.geoid10',
                 fill_color='OrRd',
                 fill_opacity=0.6,
                 line_opacity=0.2,
                 legend_name='Transit Travel Time (Min)',
                 highlight = True
                    ).add_to(m)
        
        feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)


        folium.LayerControl().add_to(m)
        m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/Transit Travel Time Maps/' + str(tod) + '/Tract_' + str(tract) + '.html')

print('Everything is Complete!')

Working on TOD 1
Working on TOD 2
Working on TOD 3
Working on TOD 4
Working on TOD 5
Everything is Complete!


In [11]:
metra_stops = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Metra Stops/Metra Stops 2019.shp', driver = 'ESRI Shapefile')
metra_stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,wheelchai,geometry
0,GENEVA,Geneva,None,41.881667,-88.310000,H,https://metrarail.com/maps-schedules/train-lin...,1,POINT (-88.31000 41.88167)
1,WCHICAGO,West Chicago,None,41.881111,-88.198889,F,https://metrarail.com/maps-schedules/train-lin...,1,POINT (-88.19889 41.88111)
2,WINFIELD,Winfield,None,41.870000,-88.156944,F,https://metrarail.com/maps-schedules/train-lin...,1,POINT (-88.15694 41.87000)
3,WHEATON,Wheaton,None,41.864444,-88.111944,E,https://metrarail.com/maps-schedules/train-lin...,1,POINT (-88.11194 41.86444)
4,COLLEGEAVE,College Ave,None,41.868333,-88.090278,E,https://metrarail.com/maps-schedules/train-lin...,1,POINT (-88.09028 41.86833)


In [12]:
cta_stops = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/CTA Stops/CTA Stops 2019.shp', driver = 'ESRI Shapefile')
cta_stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_t,parent_sta,wheelchair,geometry
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,POINT (-87.77410 41.87632)
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,POINT (-87.77132 41.87707)
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,POINT (-87.76975 41.87696)
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,POINT (-87.76745 41.87702)
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,POINT (-87.76145 41.87651)


In [13]:
cta_rail = cta_stops[~np.isnan(cta_stops['parent_sta'])]
cta_bus = cta_stops[np.isnan(cta_stops['parent_sta'])]

In [14]:
df2.TRAVEL_TIME.max()

417.3666666666667

In [19]:
    
for tract in geo.geoid10:
    print('Working on Census Tract ' + str(tract))
    m = folium.Map([41.8781, -87.6298], zoom_start=11, tiles = 'Stamen Toner')
    

        
    folium.Marker([centroids[centroids['GEOID'] == tract]['Y'].values[0], centroids[centroids['GEOID'] == tract]['X'].values[0]] , tooltip='Origin').add_to(m)

    df3 = df2.groupby(by = ['origin', 'destination'], as_index = False).mean()
    df_tract = df3[df3['origin'] == tract]

    bins = [0,15,30,60,90,df_tract.TRAVEL_TIME.max()]

    folium.Choropleth(
             geo_data=geo,
             name= str(tract),
             data=df_tract,
             columns = ['destination', 'TRAVEL_TIME'],
             key_on='feature.properties.geoid10',
             fill_color='RdYlBu',
             fill_opacity=0.6,
             line_opacity=0.2,
             legend_name='Transit Travel Time (Min)',
             highlight = True,
             bins = [float(x) for x in bins],
                ).add_to(m)
   

    #feature_group = folium.FeatureGroup(name='CTA Bus Stops', show = False)

    #for stop in cta_bus.stop_id:
    #    row = cta_bus[cta_bus['stop_id'] == stop]
    #    folium.CircleMarker([row['stop_lat'], row['stop_lon']], popup = row['stop_id'].values[0], radius = 2, fill = True, fill_color = 'purple', color = 'purple').add_to(feature_group)
    
   # m.add_child(feature_group)
    

    feature_group = folium.FeatureGroup(name='CTA Rail Stops', show = False)

    for stop in cta_rail.stop_id:
        row = cta_rail[cta_rail['stop_id'] == stop]
        folium.CircleMarker([row['stop_lat'], row['stop_lon']], popup = row['stop_id'].values[0], radius = 3, fill = True, fill_color = 'purple', color = 'green').add_to(feature_group)
    
    m.add_child(feature_group)
    
    
    feature_group = folium.FeatureGroup(name='Metra Stops', show = False)

    for stop in metra_stops.stop_id:
        row = metra_stops[metra_stops['stop_id'] == stop]
        folium.CircleMarker([row[' stop_lat'], row[' stop_lon']], popup = row['stop_id'].values[0], radius = 3, fill = True, fill_color = 'black', color = 'black').add_to(feature_group)
    
    m.add_child(feature_group)


    
    feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)

    for tract2 in centroids.GEOID:
        row = centroids[centroids['GEOID'] == tract2]
        folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)
    
    m.add_child(feature_group)
    
    
    folium.LayerControl().add_to(m)
    m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/Transit Travel Time Maps/Daily/Tract_' + str(tract) + '.html')

print('Everything is Complete!')

Working on Census Tract 17031842400.0
Working on Census Tract 17031840300.0
Working on Census Tract 17031841100.0
Working on Census Tract 17031841200.0
Working on Census Tract 17031839000.0
Working on Census Tract 17031838200.0
Working on Census Tract 17031650301.0
Working on Census Tract 17031530503.0
Working on Census Tract 17031760803.0
Working on Census Tract 17031030601.0
Working on Census Tract 17031030604.0
Working on Census Tract 17031020801.0
Working on Census Tract 17031540102.0
Working on Census Tract 17031843300.0
Working on Census Tract 17031540101.0
Working on Census Tract 17031440201.0
Working on Census Tract 17031080202.0
Working on Census Tract 17031070102.0
Working on Census Tract 17031031501.0
Working on Census Tract 17031031502.0
Working on Census Tract 17031834900.0
Working on Census Tract 17031834800.0
Working on Census Tract 17031160502.0
Working on Census Tract 17031140702.0
Working on Census Tract 17031842000.0
Working on Census Tract 17031150402.0
Working on C

Working on Census Tract 17031301801.0
Working on Census Tract 17031060800.0
Working on Census Tract 17031832300.0
Working on Census Tract 17031832500.0
Working on Census Tract 17031832000.0
Working on Census Tract 17031160601.0
Working on Census Tract 17031210501.0
Working on Census Tract 17031210601.0
Working on Census Tract 17031081300.0
Working on Census Tract 17031700502.0
Working on Census Tract 17031836100.0
Working on Census Tract 17031835700.0
Working on Census Tract 17031061100.0
Working on Census Tract 17031836400.0
Working on Census Tract 17031836500.0
Working on Census Tract 17031839600.0
Working on Census Tract 17031840000.0
Working on Census Tract 17031835600.0
Working on Census Tract 17031830800.0
Working on Census Tract 17031460301.0
Working on Census Tract 17031301701.0
Working on Census Tract 17031310400.0
Working on Census Tract 17031530100.0
Working on Census Tract 17031081100.0
Working on Census Tract 17031150700.0
Working on Census Tract 17031301702.0
Working on C

Working on Census Tract 17031420200.0
Working on Census Tract 17031420300.0
Working on Census Tract 17031440600.0
Working on Census Tract 17031440800.0
Working on Census Tract 17031440900.0
Working on Census Tract 17031560400.0
Working on Census Tract 17031450300.0
Working on Census Tract 17031671900.0
Working on Census Tract 17031672000.0
Working on Census Tract 17031680900.0
Working on Census Tract 17031681000.0
Working on Census Tract 17031240200.0
Working on Census Tract 17031240300.0
Working on Census Tract 17031200200.0
Working on Census Tract 17031410500.0
Working on Census Tract 17031410600.0
Working on Census Tract 17031170100.0
Working on Census Tract 17031170200.0
Working on Census Tract 17031170300.0
Working on Census Tract 17031381800.0
Working on Census Tract 17031170500.0
Working on Census Tract 17031351400.0
Working on Census Tract 17031381200.0
Working on Census Tract 17031381400.0
Working on Census Tract 17031390100.0
Working on Census Tract 17031430400.0
Working on C

Working on Census Tract 17031580100.0
Working on Census Tract 17031580200.0
Working on Census Tract 17031060300.0
Working on Census Tract 17031580400.0
Working on Census Tract 17031580600.0
Working on Census Tract 17031530300.0
Working on Census Tract 17031051300.0
Working on Census Tract 17031500100.0
Working on Census Tract 17031050600.0
Working on Census Tract 17031050700.0
Working on Census Tract 17031050900.0
Working on Census Tract 17031051100.0
Working on Census Tract 17031081800.0
Working on Census Tract 17031090100.0
Working on Census Tract 17031090200.0
Working on Census Tract 17031090300.0
Working on Census Tract 17031100100.0
Working on Census Tract 17031260200.0
Working on Census Tract 17031100200.0
Working on Census Tract 17031260100.0
Working on Census Tract 17031062400.0
Working on Census Tract 17031491300.0
Working on Census Tract 17031720400.0
Working on Census Tract 17031530400.0
Working on Census Tract 17031063400.0
Working on Census Tract 17031750600.0
Working on C